In [1]:
import pandas as pd
import numpy as np
from numpy import array
from sklearn.preprocessing import MinMaxScaler
from fancyimpute import KNN
from scipy import stats
import math
from random import randint
# from math import sqrt
from collections import defaultdict
from itertools import combinations
# from numpy import cov
from numpy import linalg
from numpy.linalg import eigvals

C:\Users\USER\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Loading dataset (NOMOR 1)
labels = ["Plane Weight(*10^3kg)", "Plane Velocity(km/h)", "Plane Type"]

plane_data = pd.read_csv('plane.csv', names=labels)
plane_data.head(15)

,Plane Weight(*10^3kg),Plane Velocity(km/h),Plane Type
0,26,1460,War
1,33,1450,War
2,36,1550,War
3,37,1350,War
4,72,1564,War
5,505,595,Civil
6,477,825,Civil
7,590,600,Civil


In [3]:
# Hitung Mean, Median, dan standart deviasi

P_Weight = plane_data["Plane Weight(*10^3kg)"]
# P_Weight

P_Velo = plane_data["Plane Velocity(km/h)"]
# P_Velo

P_Type = plane_data["Plane Type"]
P_Type

print ("\n----- Plane Weight ----- \n")
print(P_Weight.describe())
print ("\n----- Plane Velocity ----- \n")
print(P_Velo.describe())

print ("\n----- std from scratch ----- \n")

print ("\n----- Plane Weight ----- \n")
xs_W = P_Weight
mean_W = sum(xs_W) / len(xs_W)   # mean
var_W  = sum(pow(x-mean_W,2) for x in xs_W) / len(xs_W)  # variance
std_W  = math.sqrt(var_W)  # standard deviation
print(std_W)
print ("\n----- Plane Velocity ----- \n")
xs_V = P_Velo
mean_V = sum(xs_V) / len(xs_V)   # mean
var_V  = sum(pow(y-mean_V,2) for y in xs_V) / len(xs_V)  # variance
std_W  = math.sqrt(var_V)  # standard deviation
print(std_W )


----- Plane Weight ----- 

count      8.000000
mean     222.000000
std      252.415757
min       26.000000
25%       35.250000
50%       54.500000
75%      484.000000
max      590.000000
Name: Plane Weight(*10^3kg), dtype: float64

----- Plane Velocity ----- 

count       8.000000
mean     1174.250000
std       425.766787
min       595.000000
25%       768.750000
50%      1400.000000
75%      1482.500000
max      1564.000000
Name: Plane Velocity(km/h), dtype: float64

----- std from scratch ----- 


----- Plane Weight ----- 

236.11332025110315

----- Plane Velocity ----- 

398.26836115865393


In [4]:
# Min - Max Normalization

df1 = plane_data[["Plane Weight(*10^3kg)", "Plane Velocity(km/h)"]]
scaler = MinMaxScaler()
scaler.fit(df1)
normal = pd.DataFrame(data=scaler.transform(df1),columns=["Plane Weight(*10^3kg)", "Plane Velocity(km/h)"])
print ("\n----- Sebelum Normalisasi ----- \n")
print(plane_data)
print ("\n----- Sesudah Normalisasi ----- \n")
print(normal)


----- Sebelum Normalisasi ----- 

   Plane Weight(*10^3kg)  Plane Velocity(km/h) Plane Type
0                     26                  1460        War
1                     33                  1450        War
2                     36                  1550        War
3                     37                  1350        War
4                     72                  1564        War
5                    505                   595      Civil
6                    477                   825      Civil
7                    590                   600      Civil

----- Sesudah Normalisasi ----- 

   Plane Weight(*10^3kg)  Plane Velocity(km/h)
0               0.000000              0.892673
1               0.012411              0.882353
2               0.017730              0.985552
3               0.019504              0.779154
4               0.081560              1.000000
5               0.849291              0.000000
6               0.799645              0.237358
7               1.000000        

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [5]:
# Z-Score

# Z-score = x-mean // std

zscore = pd.DataFrame(data =stats.zscore(df1),columns=["Plane Weight(*10^3kg)", "Plane Velocity(km/h)"])

print ("\n----- Sebelum Normalisasi ----- \n")
print(plane_data)
print ("\n----- Sesudah Normalisasi ----- \n")
print(zscore)


----- Sebelum Normalisasi ----- 

   Plane Weight(*10^3kg)  Plane Velocity(km/h) Plane Type
0                     26                  1460        War
1                     33                  1450        War
2                     36                  1550        War
3                     37                  1350        War
4                     72                  1564        War
5                    505                   595      Civil
6                    477                   825      Civil
7                    590                   600      Civil

----- Sesudah Normalisasi ----- 

   Plane Weight(*10^3kg)  Plane Velocity(km/h)
0              -0.830110              0.717481
1              -0.800463              0.692372
2              -0.787757              0.943459
3              -0.783522              0.441285
4              -0.635288              0.978612
5               1.198577             -1.454421
6               1.079990             -0.876921
7               1.558574        

In [6]:
# Missing Value Imputasi K-NN K=3

# Load dataset
plane_miss_data = pd.read_csv('plane2.csv', names=labels).drop('Plane Type', axis=1)
plane_miss_data

,Plane Weight(*10^3kg),Plane Velocity(km/h)
0,26,1460.0
1,33,1450.0
2,36,1550.0
3,37,1350.0
4,72,1564.0
5,505,595.0
6,477,NaN
7,590,600.0


In [7]:
# Use 3 nearest rows which have a feature to fill in each row's missing features
X_filled_knn = KNN(k=3).fit_transform(plane_miss_data)
pd.DataFrame(data=X_filled_knn,columns=["Plane Weight(*10^3kg)", "Plane Velocity(km/h)"])

Imputing row 1/8 with 0 missing, elapsed time: 0.001


,Plane Weight(*10^3kg),Plane Velocity(km/h)
0,26.0,1460.000000
1,33.0,1450.000000
2,36.0,1550.000000
3,37.0,1350.000000
4,72.0,1564.000000
5,505.0,595.000000
6,477.0,599.632016
7,590.0,600.000000


In [8]:
# Loading Dataset (NOMOR 2)

data = np.array([5,10,11,13,15,35,50,55,72,92,204,215])
data

array([  5,  10,  11,  13,  15,  35,  50,  55,  72,  92, 204, 215])

In [9]:
#Partisi 3 Bins

bins = 3
#Equal-frequency partitioning
## Membagi kedalam 3 bins yg isinya seimbang
result = data[:(data.size)].reshape(bins, -1)
print ("\n----- Equal-frequency partitioning ----- \n")
print(result)

#Equal-width partitioning
## Membagi kedalam 3 bins yg interval data nya seimbang
print ("\n----- Equal-width partitioning ----- \n")
maxs = data.max()
mins = data.min()
width = ( maxs - mins ) // bins
interval = list()
result = list()
mydict = defaultdict(list)

for i in range(bins):
    elem = mins+width*(i+1)+1
#     interval[i].append(elem)
    interval.insert(i, elem)

for i in range(bins):
    for j in range(len(data)):
        if data[j] < interval[i]:
            if i != 0:
                if data[j]>=interval[i-1]:
                    mydict["bins",i+1].append(data[j])
            else:
                mydict["bins",i+1].append(data[j])
            
mydict
# for i in range(bins):
    


----- Equal-frequency partitioning ----- 

[[  5  10  11  13]
 [ 15  35  50  55]
 [ 72  92 204 215]]

----- Equal-width partitioning ----- 



defaultdict(list,
            {('bins', 1): [5, 10, 11, 13, 15, 35, 50, 55, 72],
             ('bins', 2): [92],
             ('bins', 3): [204, 215]})

In [10]:
# Loading Dataset (NOMOR 3)

df1

,Plane Weight(*10^3kg),Plane Velocity(km/h)
0,26,1460
1,33,1450
2,36,1550
3,37,1350
4,72,1564
5,505,595
6,477,825
7,590,600


In [11]:
# Get Means

M_W = P_Weight.mean()
M_V = P_Velo.mean()

M_Plane = np.array([[M_W, M_V]])

# original - means

sub_Plane = np.subtract(df1.values ,M_Plane)

# Covariance Matrix

Cov = np.dot(sub_Plane.T,sub_Plane) / 8

# Eigen Value & Eigen Vector

e_vals = linalg.eigvals(Cov)

print ("\n----- Mean ----- \n")
print(M_Plane)
print ("\n----- Original - Mean ----- \n")
print(sub_Plane)
print ("\n----- Covariance ----- \n")
print(Cov)
print ("\n----- Eigen Value ----- \n")
print(e_vals)
# print ("\n----- Eigen Vector ----- \n")
# print(e_vecs)


----- Mean ----- 

[[ 222.   1174.25]]

----- Original - Mean ----- 

[[-196.    285.75]
 [-189.    275.75]
 [-186.    375.75]
 [-185.    175.75]
 [-150.    389.75]
 [ 283.   -579.25]
 [ 255.   -349.25]
 [ 368.   -574.25]]

----- Covariance ----- 

[[ 55749.5    -91662.5   ]
 [-91662.5    158617.6875]]

----- Eigen Value ----- 

[  2076.6010895 212290.5864105]


In [12]:
# Loading dataset (NOMOR 4)
labels = ["Plane Weight(*10^3kg)", "Plane Velocity(km/h)", "Plane Type"]

plane_data = pd.read_csv('plane.csv', names=labels)
plane_data.head(15)

,Plane Weight(*10^3kg),Plane Velocity(km/h),Plane Type
0,26,1460,War
1,33,1450,War
2,36,1550,War
3,37,1350,War
4,72,1564,War
5,505,595,Civil
6,477,825,Civil
7,590,600,Civil


In [27]:
# Count Class

War = 0
Civil = 0

for index, row in plane_data.iterrows():
    if row['Plane Type'] == 'War':
        War +=1 # Check Class
        
    elif row['Plane Type'] == 'Civil':
        Civil +=1
#         plot2 = list(row)

print("War :",War,"\nCivi :",Civil)

# Get Minor Class

if War > Civil:
    Minor_class = "Civil"
else:
    Minor_class = "War"
    
print("Minor Class :",Minor_class)

plane_data.loc[plane_data['Plane Type'] == "Civil", 'Class'] = 'Minor'
plane_data.loc[plane_data['Plane Type'] == "War", 'Class'] = 'Major'

plane_data

# Get Distance from Minor Class
Minor = plane_data.loc[plane_data['Class'] == 'Minor']
Minor

Base_W = 477 #Set Base
Base_V = 825 #Set Base
Base = [Base_W,Base_V]


for index, row in Minor.iterrows():
    distance = math.sqrt(sum([(a - b) ** 2 for a, b in zip(row, Base)])) #Get Distance
    if distance != 0:
        Minor.loc[index,'Distance'] = distance

print(Minor)
        
# Memilih data dengan jarak terdekat

tmp = Minor.loc[Minor['Distance'].idxmin()]
W_tmp = tmp["Plane Weight(*10^3kg)"]
V_tmp = tmp["Plane Velocity(km/h)"]

val = [W_tmp,V_tmp]
print("Selected Value :",val)

# Create random
## Weight
if Base_W > W_tmp:
    print("degenerate Weight:",randint(W_tmp, Base_W))
elif Base_W < W_tmp:
    print("degenerate Weight:",randint(Base_W,W_tmp, ))
    
if Base_V > V_tmp:
    print("degenerate Weight:",randint(V_tmp, Base_V))
elif Base_V < V_tmp:
    print("degenerate Velocity:",randint(Base_V,V_tmp, ))

War : 5 
Civi : 3
Minor Class : Civil
   Plane Weight(*10^3kg)  Plane Velocity(km/h) Plane Type  Class    Distance
5                    505                   595      Civil  Minor  231.698079
6                    477                   825      Civil  Minor         NaN
7                    590                   600      Civil  Minor  251.781651
Selected Value : [505, 595]
degenerate Weight: 492
degenerate Weight: 744


C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
# Loading dataset (NOMOR 5)
labels = ["TID", "Transaction"]

sale_data = pd.read_csv('sale.csv', names=labels)
plane_data.head(15)

In [21]:
# frequent itemset with Apriori (minimum support)
